In [ ]:
import sys
sys.path.append(".")

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from models.model_S10 import Net
from utility.utils_S10 import LoadDataset, VisualizeData, LearningRateFinder, TrainTest

In [ ]:
dataloader = LoadDataset()
dataviz = VisualizeData()
lrf = LearningRateFinder()
traintest = TrainTest()

In [ ]:
train_loader,test_loader = dataloader.getData(512)

In [ ]:
dataviz.visualize(train_loader)

In [ ]:
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(3, 32, 32))

In [ ]:
lrf.findLR(model,train_loader)

In [ ]:
lrf.visualizeLR()

In [ ]:
from torch.optim.lr_scheduler import OneCycleLR

model = Net().to(device)
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

scheduler = OneCycleLR(
    optimizer,
    max_lr=0.00043,
    total_steps=24,
    epochs=24
)

EPOCHS = 24
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train_acc, train_losses = traintest.train(model, device, train_loader, optimizer, criterion)
    scheduler.step()
    test_acc, test_losses = traintest.test(model, device, test_loader, criterion)

In [ ]:
traintest.visualizeLoss(train_losses,train_acc,test_losses,test_acc)